# Exercise 3.3, part 2

Import the pySpark libraries

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge pyspark

Solving environment: done

# All requested packages already installed.



In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, window, to_timestamp
from pyspark.sql.types import StructType, StructField, StringType

Launch a Spark session

In [5]:
spark = SparkSession.builder.appName('Packt').getOrCreate()

Get the raw data from a local socket

In [6]:
raw_stream = spark.readStream.format('socket').option('host', 'localhost').option('port', 1234).load()

Set up the Twitter date-time format

In [4]:
tweet_datetime_format = 'EEE MMM dd HH:mm:ss ZZZZ yyyy'

Create the schema to parse a tweet in JSON format; we only need two columns

In [3]:
schema = StructType([StructField('created_at', StringType(), True),
                     StructField('text', StringType(), True)])

Parse the json to get separate fields

In [7]:
tweet_stream = raw_stream.select(from_json('value', schema).alias('tweet'))

Create a timestamp by parsing the created_at field

In [8]:
timed_stream = tweet_stream.select(
    to_timestamp('tweet.created_at', tweet_datetime_format).alias('timestamp'),
    'tweet.text')

Create a sliding window of 1 minute with a slide of 10 seconds, with a 'slack time' of 2 seconds

In [9]:
windowed = timed_stream \
    .withWatermark('timestamp', '2 seconds') \
    .groupBy(window('timestamp', '1 minute', '10 seconds'))

Count the tweets per window

In [10]:
counts_per_window = windowed.count().orderBy('window')

output the windows and counts to the console

In [ ]:
query = counts_per_window.writeStream.outputMode('complete').format('console').option("truncate", False).start()
query.awaitTermination()